In [13]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [14]:
df = pd.read_csv('insurance.csv')

In [15]:
df.sample(5)

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
80,56,95.8,1.67,50.00,False,Jalandhar,unemployed,High
48,36,94.8,1.66,32.69,True,Chennai,unemployed,Medium
44,59,77.0,1.60,50.00,True,Lucknow,private_job,Medium
82,35,56.0,1.77,12.96,False,Delhi,unemployed,Low
95,36,52.8,1.57,19.64,False,Indore,business_owner,Low


In [35]:
df['occupation'].unique()

array(['retired', 'freelancer', 'student', 'government_job',
       'business_owner', 'unemployed', 'private_job'], dtype=object)

In [16]:
df_feat = df.copy()

In [17]:
## Feature 1: BMI
df_feat["bmi"] = df_feat["weight"] + (df_feat["height"] ** 2)

In [18]:
## Feature 2: Age Group
def age_group(age):
    if age < 25:
        return "young"
    elif age < 45:
        return "middle_aged"
    return "senior"

df_feat["age_group"] = df_feat["age"].apply(age_group)

In [19]:
## Feature 3: Lifestyle Risk
def lifestyle_risk(row):
    if row["smoker"] and row["bmi"] > 30:
        return "high"
    elif row["smoker"] or row["bmi"] > 27:
        return "medium"
    return "low"

df_feat["lifestyle_risk"] = df_feat.apply(lifestyle_risk, axis=1)

In [20]:
tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"
]

In [21]:
## Feature 4: City Tier
def city_tier(city):
    if city in tier_1_cities:
        return 1
    elif city in tier_2_cities:
        return 2
    else:
        return 3

In [22]:
df_feat["city_tier"] = df_feat["city"].apply(city_tier)

In [23]:
df_feat.drop(columns=['age', 'weight', 'height', 'smoker', 'city'])[['income_lpa', 'occupation', 'bmi', 'age_group', 'lifestyle_risk', 'city_tier', 'insurance_premium_category']].sample(5)

,income_lpa,occupation,bmi,age_group,lifestyle_risk,city_tier,insurance_premium_category
48,32.69,unemployed,97.5556,middle_aged,high,1,Medium
39,11.99,unemployed,103.4224,senior,high,1,High
12,17.58,freelancer,98.3684,middle_aged,high,2,High
36,0.53,retired,61.0896,senior,medium,1,Medium
42,2.60,student,73.1041,young,high,2,Medium


In [25]:
## Select Feature and Target
X = df_feat[["bmi", "age_group", "lifestyle_risk", "city_tier", "income_lpa", "occupation"]]
y = df_feat["insurance_premium_category"]

In [26]:
X

,bmi,age_group,lifestyle_risk,city_tier,income_lpa,occupation
0,122.2336,senior,medium,2,2.92000,retired
1,104.4489,middle_aged,medium,1,34.28000,freelancer
2,59.4896,middle_aged,medium,2,36.64000,freelancer
3,111.8025,young,high,1,3.34000,student
4,64.7600,senior,high,2,3.94000,retired
...,...,...,...,...,...,...
95,55.2649,middle_aged,medium,2,19.64000,business_owner
96,116.1716,middle_aged,medium,1,34.01000,private_job
97,64.0400,senior,medium,1,44.86000,freelancer
98,104.4124,middle_aged,medium,1,28.30000,business_owner


In [27]:
y

0       High
1        Low
2        Low
3     Medium
4       High
       ...  
95       Low
96       Low
97       Low
98       Low
99       Low
Name: insurance_premium_category, Length: 100, dtype: object

In [28]:
## Define categorical and numeric features
categorical_features = ["age_group", "lifestyle_risk", "occupation", "city_tier"]
numeric_features = ["bmi", "income_lpa"]

In [29]:
## Create column transformer for OHE
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), categorical_features),
        ("num", "passthrough", numeric_features)
    ]
)
 

In [30]:
## Create a pipeline with preprocessing and random forest classifier
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [31]:
## Split data and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['age_group',
                                                   'lifestyle_risk',
                                                   'occupation', 'city_tier']),
                                                 ('num', 'passthrough',
                                                  ['bmi', 'income_lpa'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [32]:
## Predict and evaluate
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.9

In [33]:
X_test.sample(5)

,bmi,age_group,lifestyle_risk,city_tier,income_lpa,occupation
81,85.1921,middle_aged,high,2,22.19,freelancer
39,103.4224,senior,high,1,11.99,unemployed
80,98.5889,senior,medium,2,50.00,unemployed
31,54.4489,middle_aged,high,2,11.77,private_job
33,70.5976,senior,medium,1,1.46,retired


In [34]:
import pickle

# Save the trained pipeline using pickle
pickle_model_path = "model.pkl"
with open(pickle_model_path, "wb") as f:
    pickle.dump(pipeline, f)